<a href="https://colab.research.google.com/github/wangbxj1234/voxel/blob/main/collate_fn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import torch

In [42]:
def collate_fn(coord, feat, label):#(B,N,C)
    #coord, feat, label = list(zip(*batch))#(B,N,C)
    offset = [6,12]
    
    ####
    #coord, feat, label = torch.cat(coord,dim=0), torch.cat(feat,dim=0), torch.cat(label,dim=0) #但是属于不同batch的，就算坐标近也不能分进一个v。
    offset = torch.IntTensor(offset)##(N,3),(N,C),(N,),O

    ##to np
    coord, feat, label = coord.cpu().detach().numpy(),feat.cpu().detach().numpy(),label.cpu().detach().numpy()
    ##
    PN=0;
    voxel=[]
    for o in offset:
        c, f, l =coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]#(n,C),(n,C),(n,)
        discrete_coord = np.floor(c *4) 
        assert discrete_coord.ndim == 2
    # Floor first for negative coordinates
        arr = discrete_coord.copy()
        arr = arr.astype(np.uint64, copy=False)
        hashed_arr = np.uint64(14695981039346656037) * np.ones(arr.shape[0], dtype=np.uint64)
        for j in range(arr.shape[1]):
            hashed_arr *= np.uint64(1099511628211)
            hashed_arr = np.bitwise_xor(hashed_arr, arr[:, j])
        idx_sort = np.argsort(hashed_arr) 
        key_sort = hashed_arr[idx_sort] 
        _, count1 = np.unique(key_sort, return_counts=True)
        coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]= c[idx_sort],f[idx_sort],l[idx_sort]
        voxel.append(count1)
        PN=int(o)

    ##to torch
    coord, feat, label = torch.tensor(coord),torch.tensor(feat),torch.tensor(label)
    ##    

    return coord, feat, label, offset, voxel

In [54]:
#coord, feat, label=torch.rand(2,6,3),torch.rand(2,6,4),torch.rand(2,6,1)
coord, feat, label=torch.rand(2*6,3),torch.rand(2*6,4),torch.rand(2*6,1)
coord[0,:]=coord[1,:]
coord

tensor([[0.7676, 0.7569, 0.8271],
        [0.7676, 0.7569, 0.8271],
        [0.7305, 0.9419, 0.2595],
        [0.1547, 0.1996, 0.9441],
        [0.8287, 0.0388, 0.0280],
        [0.0960, 0.8266, 0.1421],
        [0.8201, 0.9880, 0.8288],
        [0.1124, 0.0362, 0.3869],
        [0.7995, 0.6658, 0.4634],
        [0.2010, 0.6069, 0.8736],
        [0.5766, 0.6010, 0.2131],
        [0.2119, 0.9680, 0.4654]])

In [55]:
coord, feat, label, offset, voxel = collate_fn(coord, feat, label)
print(coord.size(),feat.size(),label.size())

torch.Size([12, 3]) torch.Size([12, 4]) torch.Size([12, 1])


In [56]:
voxel

[array([1, 2, 1, 1, 1]), array([1, 1, 2, 1, 1])]

In [58]:
voxel2=np.concatenate(voxel, axis=0 )# 6-1=5
torch.tensor(voxel2)

tensor([1, 2, 1, 1, 1, 1, 1, 2, 1, 1])